<a href="https://colab.research.google.com/github/RuslanFatkhutdinov/google-analytics-api-to-pandas-df/blob/master/google-analytics-api-to-pandas-df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Получение данных из Google Analytics по API.

---

## О тетрадке
В тетрадке собран алоритм экспорта данных по API Google Analtics при помощи модуля gaapi4py. 

## Структура
1. Установка и импорт необходимых модулей.
2. Подключение к Google Drive.
3. Авторизация в Google Analytics.
4. Экспорт данных:
    - Простой запрос;
    - Запрос с обходом семплирования.
5. Сохранение данных в CSV.

## Необходимые модули
- **gaapi4py** — для работы с API Google Analytics;
- **join** из **os.path** — присоединяет части адреса в операционной системы;
- **drive** из **google.colab** — чтение и запись данных на Google Drtive;
- **pandas** — работа с данными в таблицах;
- **date** из **datetime** — оставить дату из объекта с датой и временем;
- **timedelta** из **datetime** — "смещение" даты;
- **sleep** из **time** — таймауты между запросами.


## Полезные ссылки
- [Документация по модулю gaapi4py](https://pypi.org/project/gaapi4py/);
- [Dimensions & Metrics Explorer](https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/);
- [Google API Console](https://console.developers.google.com/apis/dashboard);
- [Google Analytics](https://analytics.google.com/analytics/).

# 1. Установка и импорт необходимых модулей

Модуля gaapi4py в colab нет. Надо устанавливать каждый перезапуск виртуальной машины `¯\_(ツ)_/¯`.

In [0]:
!pip install gaapi4py

Импортируем модули

In [0]:
from os.path import join 
from google.colab import drive 
import pandas as pd
from gaapi4py import GAClient
from datetime import date, timedelta
from time import sleep

# 2. Подключение к Google Drive

Чтобы работать с файлами, хранящимися в Google Drive, в том числе и сохранять файлы на Google Drive, подключаемся к его файловой системе и сохраняем в переменной ROOT путь к "рабочей" области.

In [0]:
ROOT = '/content/drive'
drive.mount(ROOT)

В левой части тетрадки есть "ушко", нажав на которое откроется дополнительное меню. На вкладке **Files** показана стркутура Google Drive. Можно сразу перейти по пути **/contetnt/drive/My Drive** чтобы посмотреть структуру диска.

![alt text](https://d1ro8r1rbfn3jf.cloudfront.net/ms_2876/dd5ONjtUan25JSZmy6YMjHwztm6SpM/2019-10-01%2B14-54-36.jpg?Expires=1569931200&Signature=qXH6QWuhrw2gFCj585V8-e2jC1DN0B8hfsoLzD3u~S-d03UJhgaPC-EI7v3mmQ-TQtxajAC0Bz5Oa7OZTRuiiVeVQA4JYbInEF1aMvN2l8ch0biYlbEX3AdhPajRsZH9cZ9CxWRkf1nZdmLqVeHaIEQMu-KVEYLgGtT2cNvWmpzHRbeaGWK1F7RqnoSh-eX2rgUDfuIdP0CrWBNEkhrVrw8Pp5g~zaYQeTX1WJ4I~1JpY6aiokPH45GS5ALjO~j0ucCYJjuBk0O2kIew2TSl20mV6IWrQX1B7esI2e6KBaVTrLDDNORNTRn9PnFMdYxG8kOQx1Ikc~HgV9oU2ZxJ1g__&Key-Pair-Id=APKAJBCGYQYURKHBGCOA)

# 3. Авторизация в Google Analytics
1. Получаем сервисный ключ. Как это сделано опиасно в [этой статье](https://medium.com/@fatkhutdinov/%D1%8D%D0%BA%D1%81%D0%BF%D0%BE%D1%80%D1%82-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85-%D0%B8%D0%B7-api-google-analytics-%D0%B2-pandas-dataframe-2b045cf47f20).
2. Даем доступ сервисному аккаунту к нужному представлению в Google Analytics.
2. Сохраняем json файл с ключем в папке Google Drive (в случае с этой тетрадкой файл будет сохранен по пути **'My Drive/Workspace/Google Account Auth'**).

- Сохраняем путь к файлу с ключем в переменной **AUTH_KEY**.
- Методом **join** объединяем путь к рабочей области (переменная **ROOT**) и к сервисному ключу (переменная **AUTH_KEY**) и сохраняем в переменной **KEY_PATH**.

In [0]:
AUTH_KEY = 'My Drive/Workspace/Auth/auth_key.json'
KEY_PATH = join(ROOT, AUTH_KEY)

# 4. Экспорт данных
В зависимости от того, начинает GA семплироват данные или нет, можно использовтаь разные запросы:
- Простой запрос;
- Запрос с обходом семплирования.

Сохраняем "авторизацию" в переменной **ga**.

In [0]:
ga = GAClient(KEY_PATH)

## 4.1. Простой запрос

In [0]:
ga.set_view_id('XXX') # Указываем ID представления
ga.set_dateranges('2019-09-30', '2019-09-30') # Указываем даты начала и конца

# Формируем список параметров и показателй
request_body = {
    'dimensions': {
        'ga:transactionId',
        'ga:date',
        'ga:clientID'
        },
    'metrics': {
        'ga:transactionRevenue'
        }
    }

# Сохраняем полученные данные в переменную
response = ga.get_all_data(request_body)

Быстрый доступ к информации:
- **response['info']** — мета-информация,
- **response['data']** — датафрейм для pandas.

# 4.2. Запрос с обходом семплирования

In [0]:
ga.set_view_id('XXX') # ID представления
start_date = date(2018,9,1) # Дата начала (Г,М,Д)
end_date = date(2019,9,29) # Дата завершения

# Цикл получения данных
df_list = []
iter_date = start_date
while iter_date <= end_date:
    ga.set_dateranges(iter_date, iter_date)
    response = ga.get_all_data({
        'dimensions': {
            'ga:transactionId',
            'ga:date',
            'ga:clientID'
        },
        'metrics': {
            'ga:transactionRevenue'
        }
    })
    df = response['data']
    df['date'] = iter_date
    df_list.append(response['data'])
    iter_date = iter_date + timedelta(days=1)
    sleep(1)

# Сохраняем полученные данные
all_data = pd.concat(df_list, ignore_index=True)

# 5. Сохраняем данные в CSV

В скобках в кавычках указываем путь (с учетом того, что путь **'/content/drive/My Drive/'** — путь до корня рабочей области), название и расширение `.csv`.

Код для сохранения данных простого запроса.

In [0]:
response['data'].to_csv('/content/drive/My Drive/Workspace/response.csv', index = False)

Код для сохранения данных запроса с обходом семплирования.

In [0]:
all_data.to_csv('/content/drive/My Drive/Workspace/all_data.csv', index = False)